# Retrieval
Add context to the LLM

In [3]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("pdfs/A00041-00045.pdf")
loaded = loader.load()

In [4]:
type(loaded)

list

In [5]:
loaded[0]

Document(page_content='BOE núm. 1 Lunes 1 enero 2001 41\nIII. Otras disposiciones\nCORTES GENERALES\n24 ACUERDO de 19 de diciembre de 2000, de la Mesa del Sena-\ndo, referente a la convocatoria de becas de formación sobre\ncomunicación institucional relacionada con el Senado.\nEl Senado, por acuerdo de la Mesa de 19 diciembre de 2000, convoca\ntres becas individuales de formación sobre comunicación institucional rela-\ncionada con el Senado, de acuerdo con las bases que figuran a continuación:\nPrimera. Objeto.— El Senado convoca tres becas individuales para rea-\nlizar actividades de formación teórico-práctica sobre comunicación ins-titucional relacionada con la actividad parlamentaria que determine el\nDepartamento de Prensa de la Cámara.\nSegunda. Condiciones de las becas.— 1. Cada una de las tres becas\nse disfrutará desde su otorgamiento por la Mesa hasta el 31 de diciembre\nde 2001 y su cuantía mensual será de 107.083 pesetas brutas, que se\npercibirán finalizado cada mes.\n2. El

In [6]:
loaded[1]

Document(page_content='42 Lunes 1 enero 2001 BOE núm. 1\n', metadata={'source': 'pdfs/A00041-00045.pdf', 'page': 1})

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(loaded)

In [13]:
import time
start = time.time()
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OllamaEmbeddings())
end = time.time()
embedding_time = end - start
print("Time taken:", embedding_time)

Time taken: 23.917999744415283


Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado

In [14]:
import os
len(os.listdir("pdfs"))

4313

In [16]:
print(f"El tiempo que tomaria hacer embeddings de todos los documentos es de {len(os.listdir('pdfs')) * embedding_time / 60} minutos que son {len(os.listdir('pdfs')) * embedding_time / 3600} horas")

El tiempo que tomaria hacer embeddings de todos los documentos es de 1719.3055482943853 minutos que son 28.655092471573088 horas


In [17]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?")

docs

[Document(page_content='todo lo que se refiere a los asuntos, hechos o informaciones de los que\nhubiera tenido conocimiento durante el período de la beca. No deberán\ncomunicar bajo ninguna forma documentos o informaciones que no hubie-\nran sido hechos públicos a personas que no estuvieren cualificados para\ntener conocimientos de los mismos, a reserva del acuerdo previo de la\nInstitución. Seguirán sometidos a esta obligación después de finalizar su\nbeca.', metadata={'page': 0, 'source': 'pdfs/A00041-00045.pdf'}),
 Document(page_content='todo lo que se refiere a los asuntos, hechos o informaciones de los que\nhubiera tenido conocimiento durante el período de la beca. No deberán\ncomunicar bajo ninguna forma documentos o informaciones que no hubie-\nran sido hechos públicos a personas que no estuvieren cualificados para\ntener conocimientos de los mismos, a reserva del acuerdo previo de la\nInstitución. Seguirán sometidos a esta obligación después de finalizar su\nbeca.', metadata={

In [18]:
from langchain_community.llms import Ollama
llm = Ollama(model="llama2")

In [19]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """
Answer the user's questions based on the below context. 
If the context doesn't contain any relevant information to the question, don't make something up and just say "I don't know":

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            SYSTEM_TEMPLATE,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

In [ ]:
from langchain_core.messages import HumanMessage

document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="Cuál es la cuantía mensual bruta de las becas de formación sobre comunicación institucional relacionada con el Senado?")
        ],
    }
)

'Based on the provided context, the monthly stipend for each of the three fellowships is 107.083 pesetas brutas.'